In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glaive-python-code-qa-dataset/train.csv
/kaggle/input/gemma/keras/gemma_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_2b_en/2/assets/tokenizer/vocabulary.spm


# Introduction 

In this notebook, you will discover effective fine-tuning strategies to enhance the performance of the Google Gemma Large Language Model (LLM) on a targeted task.

The task we'll be fine-tuning Gemma for is answering common questions about the Python programming language. Gemma, having undergone training on a vast dataset, has developed a strong grasp of statistical language patterns, positioning it as a proficient reasoning engine within the Large Language Model (LLM) domain.
Through fine-tuning, we aim to:

- Tailor the model to excel in a specific use case: answering common questions about Python.
- Ensure that the model's outputs align with the expectations of this use case.
- Minimize the likelihood of model hallucinations by ensuring that its outputs are both helpful and accurate.

# What Is Fine-Tuning?

Pre-trained language models demonstrate vast abilities, they lack expertise in specific tasks by default. Although they possess a strong understanding of language, developers refine their performance through a process known as LLM fine-tuning. This process involves optimizing their capabilities for tasks such as sentiment analysis, language translation, or addressing queries within specialized domains. Fine-tuning these models is essential for unlocking their full potential and tailoring them to suit specific applications.

Fine-tuning can be likened to giving a final touch to these versatile models. Imagine having a friend who excels in various fields but needs to master a specific skill for a particular occasion. You would provide them with focused training in that area, wouldn't you? Similarly, during fine-tuning, developers provide targeted training to pre-trained language models to enhance their effectiveness in specific tasks.


# Determining When to Utilize Fine-Tuning:

When developing an LLM application, the initial step involves selecting an appropriate pre-trained or foundational model suitable for our specific use case. Subsequently, employing prompt engineering allows us to promptly assess whether the chosen model realistically fits our intended purpose and evaluate its performance.

If prompt engineering fails to yield a satisfactory level of performance, fine-tuning becomes necessary. fine-tuning is advisable when aiming to tailor the model to excel in a particular task or set of tasks, provided there exists a labeled, unbiased, and diverse dataset. Additionally, fine-tuning is recommended for domain-specific applications. In this case, we are fine-tuning for the Python question and answering task.

In this notebook, you will learn how to fine-tune the Google Gemma open LLM model to answer questions related to the Python programming language. Follow the steps below to fine-tune Gemma.

# Step 1: Setup Gemma

To use Gemma in your kaggle notebook, request access on Kaggle:

1. Sign in kaggle.com
2. Open the [Gemma model card](https://www.kaggle.com/models/google/gemma/frameworks/keras) and request access

### Install dependencies 
Install Keras, KerasNLP, we will use keras to fine tune the model. 


In [2]:
# Install and upgrade the keras-nlp library
!pip install -q -U keras-nlp

# Install and upgrade Keras to version 3 or higher
!pip install -q -U keras>=3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.


### Choose a Backend:

Keras, a user-friendly high-level deep learning API, is crafted for simplicity and seamless usage across multiple frameworks. With Keras 3, you have the flexibility to execute workflows on one of three backends: TensorFlow, JAX, or PyTorch.
In this tutorial, we'll configure the backend to utilize JAX.

In [3]:
import os

# Set the Keras backend to "jax" (or "torch" or "tensorflow").
os.environ["KERAS_BACKEND"] = "jax"

# Ensure optimal memory usage on the JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

### Import packages 

In [4]:
import keras
import keras_nlp

2024-03-14 00:55:35.673331: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 00:55:35.673523: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 00:55:35.850919: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Load Gemma model with keras_nlp

In [5]:
# Load the GemmaCausalLM model using the specified preset "gemma_2b_en".
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

# Display a summary of the GemmaCausalLM model architecture.
gemma_lm.summary()

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

## Inference with Gemma before Fine-Tuning
Let's attempt to pose a Python-related question to the Gemma model before fine-tuning it with our dataset, to observe how Gemma responds to Python queries.

In [6]:
# Define a function to format a question into a prompt template.
def ask_question(query:str)->str:
    """
    Formats the input query into a prompt template.
    
    Args:
        query (str): The question to be asked.
        
    Returns:
        str: The formatted prompt containing the question and answer.
    """
    template = "Question:\n{question}\n\nAnswer:\n{answer}"
    prompt = template.format(
        question=query,
        answer="",
    )
    return prompt

# Define a TopKSampler object with a seed value of 2
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)

# Compile the GemmaCausalLM model with the defined sampler
gemma_lm.compile(sampler=sampler)

In [7]:
# Define the prompt by asking a question about Flask and Flask-Restful issue with JSON arguments for a POST request.
prompt = ask_question("I am new to Flask and Flask-Restful and I'm having an issue with JSON arguments for a POST request, they are getting set to NONE and not working. I am able to take arguments from the form-data, using the POSTMAN plugin for chrome. But, when I switch to raw and feed a JSON, it fails to read the JSON and assigns a NONE to all my arguments. I am using python-2.6, Flask-Restful-0.3.3, Flask-0.10.1, Chrome, POSTMAN on Oracle Linux 6.5. How can I solve this issue?")

# Generate the response from Gemma based on the provided prompt.
print(gemma_lm.generate(prompt, max_length=512))

Question:
I am new to Flask and Flask-Restful and I'm having an issue with JSON arguments for a POST request, they are getting set to NONE and not working. I am able to take arguments from the form-data, using the POSTMAN plugin for chrome. But, when I switch to raw and feed a JSON, it fails to read the JSON and assigns a NONE to all my arguments. I am using python-2.6, Flask-Restful-0.3.3, Flask-0.10.1, Chrome, POSTMAN on Oracle Linux 6.5. How can I solve this issue?

Answer:
If you use Flask and Flask-Restful, you can use the <em><strong>jsonify</strong></em> function to send JSON data to a RESTful web server.

Example:

import flask
from flask import jsonify
app = flask.Flask(__name__)

@app.route('/test', methods=['PUT', 'POST'])
def test():
    json_data = request.json
    # do whatever processing you need here
return jsonify({"message": "test"})

In the above example, the function <em><strong>test()</strong></em> is defined as a RESTful endpoint that receives a <strong>POST</stro

In [8]:
# Define the prompt asking about installing Python 3 on an AWS EC2 instance using sudo yum, encountering an error message, and searching online for a solution.
prompt = ask_question("How can I install Python 3 on an AWS EC2 instance? I tried using the command `sudo yum install python3`, but I received an error message saying `No package python3 available.`. I searched online but didn't find a solution. Do I need to download and install it manually?")

# Generate the response from Gemma based on the provided prompt.
print(gemma_lm.generate(prompt, max_length=512))


Question:
How can I install Python 3 on an AWS EC2 instance? I tried using the command `sudo yum install python3`, but I received an error message saying `No package python3 available.`. I searched online but didn't find a solution. Do I need to download and install it manually?

Answer:
You can install the latest version of Python 3 from the Amazon Linux repository using the following command:
$ sudo yum -y install python3

This will install Python 3.6.8 and Python 3.7.3 in the /usr/local/bin/python directory. You can also install Python 3.9 using the following command:
$ sudo yum -y install python39


# Step 2: Load and prepare dataset

The dataset comprises approximately 140,000 code problems and solutions meticulously curated to develop intelligent Python code assistants. Presented in a question-and-answer format, this dataset features genuine user queries tailored for coding challenges, spanning from fundamental data type concepts to intricate object-oriented programming scenarios and functionalities. Notably, Python code constitutes around 60% of the dataset, ensuring a comprehensive coverage of code related inquiries.
[Data source](https://huggingface.co/datasets/glaiveai/glaive-code-assistant)

In [9]:
# Load the dataset from the specified CSV file path
dataset = pd.read_csv("/kaggle/input/glaive-python-code-qa-dataset/train.csv")

# Display the first 10 rows of the dataset
dataset.head(10)

,answer,question
0,"Yes, you can format the output text in Bash to...",How can I output bold text in Bash? I have a B...
1,"To install Python 3 on an AWS EC2 instance, yo...",How can I install Python 3 on an AWS EC2 insta...
2,You can achieve the desired time format using ...,How can I format the elapsed time from seconds...
3,Your current implementation is actually quite ...,I am trying to create a matrix of random numbe...
4,The use of 'self' in Python is quite different...,I am learning Python and have noticed extensiv...
5,Gradient clipping is a technique to prevent ex...,What is the correct method to perform gradient...
6,The error is due to pip trying to connect over...,I am a Python beginner and I'm trying to use p...
7,Fuzzy matching is possible with Python pandas ...,How can I perform a fuzzy match merge using Py...
8,The issue you're experiencing is due to the wa...,I am new to Flask and Flask-Restful and I'm ha...
9,"In sklearn, the Logistic Regression model is i...",How can I find the weight vector 'w' in Binary...


In [10]:
# Retrieve the question from the dataset located at index 1 to see how a question looks
dataset["question"].iloc[1]

"How can I install Python 3 on an AWS EC2 instance? I tried using the command `sudo yum install python3`, but I received an error message saying `No package python3 available.`. I searched online but didn't find a solution. Do I need to download and install it manually?"

In [11]:
# Retrieve the answer from the dataset located at index 1 to see the answer
dataset["answer"].iloc[1]

"To install Python 3 on an AWS EC2 instance, you can use the Amazon Linux Extras Library. This library is a curated set of software that Amazon provides for the Amazon Linux 2 platform. It includes newer versions of software, like Python 3, that are not included in the default Amazon Linux 2 repositories. Here is a step by step process on how to do it:\n\n1. First, update your instance with the following command:\n\n```bash\nsudo yum update -y\n```\n\n2. Next, list available packages in the Amazon Linux Extras repository by typing:\n\n```bash\nsudo amazon-linux-extras list\n```\n\n3. You should see python3.8 available in the list. To install it, use the following command:\n\n```bash\nsudo amazon-linux-extras install python3.8\n```\n\n4. Verify the installation by checking the Python version:\n\n```bash\npython3 --version\n```\n\nThis should return something like:\n\n```bash\nPython 3.8.8\n```\n\nThat's it! You have now installed Python 3 on your AWS EC2 instance."

# Step 3: LoRA Fine-tuning

## What is LoRA Fine-tuning 

LoRA Fine-tuning (Low Rank Adaptation) presents a fine-tuning methodology tailored for Large Language Models (LLMs) such as Gemma. This technique streamlines the training process by constraining the number of trainable parameters. It achieves this by immobilizing the existing model weights while introducing a reduced set of new weights. This optimization not only accelerates training speed but also enhances memory efficiency. Moreover, it results in more concise model weights while preserving the model's ability to generate high-quality outputs.

## Why LoRA Fine-tuning for Fine-tuning LLMs to Answer Python Questions and Answers?

In fine-tuning LLMs to address Python-related queries, the adoption of LoRA Fine-tuning proves invaluable. By optimizing the training process and minimizing the computational overhead, LoRA Fine-tuning ensures efficient utilization of resources. This, in turn, facilitates the seamless adaptation of the model to the nuances of Python-related inquiries, ultimately enhancing the accuracy and effectiveness of the model's responses.

In [12]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [13]:
# Iterate over each row in the DataFrame and construct the formatted strings
data = []
for index, row in dataset.iterrows():
    data.append(f"Question:\n{row['question']}\n\nAnswer:\n{row['answer']}")
# Select only first 50 of the dataset to reduce training time
data = data[:50]

In [14]:
# Limit the input sequence length to 128 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 128
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

gemma_lm.fit(data, epochs=1, batch_size=1)

50/50 ━━━━━━━━━━━━━━━━━━━━ 1001s 19s/step - loss: 1.9054 - sparse_categorical_accuracy: 0.5807


# Step 4: Infrencing With Gemma

In [15]:
# Define the prompt by asking a question about Flask and Flask-Restful issue with JSON arguments for a POST request.
prompt = ask_question("I am new to Flask and Flask-Restful and I'm having an issue with JSON arguments for a POST request, they are getting set to NONE and not working. I am able to take arguments from the form-data, using the POSTMAN plugin for chrome. But, when I switch to raw and feed a JSON, it fails to read the JSON and assigns a NONE to all my arguments. I am using python-2.6, Flask-Restful-0.3.3, Flask-0.10.1, Chrome, POSTMAN on Oracle Linux 6.5. How can I solve this issue?")

# Generate the response from Gemma based on the provided prompt.
print(gemma_lm.generate(prompt, max_length=512))

Question:
I am new to Flask and Flask-Restful and I'm having an issue with JSON arguments for a POST request, they are getting set to NONE and not working. I am able to take arguments from the form-data, using the POSTMAN plugin for chrome. But, when I switch to raw and feed a JSON, it fails to read the JSON and assigns a NONE to all my arguments. I am using python-2.6, Flask-Restful-0.3.3, Flask-0.10.1, Chrome, POSTMAN on Oracle Linux 6.5. How can I solve this issue?

Answer:
The issue is that you are using Flask-Restful-0.3.3 and not the latest version 0.5.3.

Flask-Restful 0.5.3 was released on June 12th, 2013. Flask-Restful 0.3.3 was released on June 25th, 2011.

You are using a very old version of Flask-Restful.

The problem is that 0.3.3 doesn't support JSON arguments.

The latest release, 0.5.3, does support JSON arguments.

To fix the issue, install the 0.5.3 release.

To install the 0.5.3 release, run: pip install Flask-Restful==0.5.3

Note: The 0.5.3 release is not compatible

In [16]:
# Define the prompt asking about installing Python 3 on an AWS EC2 instance using sudo yum, encountering an error message, and searching online for a solution.
prompt = ask_question("How can I install Python 3 on an AWS EC2 instance? I tried using the command `sudo yum install python3`, but I received an error message saying `No package python3 available.`. I searched online but didn't find a solution. Do I need to download and install it manually?")

# Generate the response from Gemma based on the provided prompt.
print(gemma_lm.generate(prompt, max_length=512))


Question:
How can I install Python 3 on an AWS EC2 instance? I tried using the command `sudo yum install python3`, but I received an error message saying `No package python3 available.`. I searched online but didn't find a solution. Do I need to download and install it manually?

Answer:
To install Python 3 on AWS EC2 instances, follow these steps:


1. Create a new AMI from one of your EC2 instances that already has Python 3 installed.
2. Create a new Amazon Machine Image (AMI) from the new AMI.
3. Launch a new EC2 instance from the new AMI.
4. After the EC2 instance is launched, you can use the AWS console or the AWS CLI to install Python 3 on the EC2 instance.

Note: You can also use the Amazon Elastic Container Service (Amazon ECS) to launch Python 3-based Docker containers.


## Observations

Fine-tuning has undoubtedly enhanced Gemma's performance, resulting in improved Python-related results. 

To further enhance the performance of the fine-tuned Gemma on Python-related queries, several steps can be taken:

- **Refine the Fine-tuning Dataset:** Ensure the fine-tuning dataset is diverse and representative of the questions users commonly ask. Include a wide range of Python-related queries to cover various scenarios adequately.

- **Adjust Fine-tuning Parameters:** Experiment with different fine-tuning parameters to optimize performance. Fine-tune the model with varying learning rates, batch sizes, and training epochs to find the optimal configuration.

- **Select Appropriate Fine-tuning Objective:** Choose a fine-tuning objective that aligns with the specific task at hand. Consider objectives tailored to enhancing the model's performance on Python-related queries.

- **Mitigate Overfitting:** Apply techniques to prevent overfitting during fine-tuning, such as regularization or dropout. Ensure the fine-tuned model maintains a balance between specialization and generalization.

- **Expand Fine-tuning Dataset:** Fine-tune the model on a larger and more diverse dataset containing a broad range of Python-related questions and answers. A more extensive dataset can help the model better understand the intricacies of Python programming and improve its performance on related queries.

By addressing these considerations and fine-tuning strategies, Gemma's performance can be further enhanced, leading to more accurate and informative responses to Python-related queries.



# Conclusion

In this notebook, we explored the process of fine-tuning Gemma, a Google Language Model (LLM), using a dataset of Python questions and answers. Through this endeavor, we observed a notable improvement in Gemma's ability to respond effectively to Python-related queries.

Fine-tuning Gemma proved to be a valuable exercise, as it resulted in enhanced performance and accuracy in addressing Python-related inquiries. By leveraging a tailored dataset and fine-tuning techniques, Gemma's responses became more insightful and informative, providing users with a better overall experience.

Overall, the process of fine-tuning Gemma underscored the importance of customization and optimization in natural language processing tasks. Through continued refinement and adaptation, Gemma can continue to evolve and deliver even more precise and helpful responses to Python questions in the future.